# Exploration Notebook: 기준금리와 환율 관계 탐색

본 노트북은 데이터 수집 후, **데이터 품질 검증 → 탐색적 데이터 분석(EDA) → 1차 분석** 과정을 정리합니다.  

---

## 1. 데이터 로드
```python
import sqlite3
import pandas as pd

# DB 연결
conn = sqlite3.connect("../data/economic.db")

# 금리와 환율 데이터 불러오기
interest = pd.read_sql("SELECT * FROM interest_rate", conn)
exchange = pd.read_sql("SELECT * FROM exchange_rate", conn)

conn.close()

print("Interest Rate:", interest.shape)
print("Exchange Rate:", exchange.shape)

interest.head()
```

---

## 2. 데이터 기본 탐색

- 데이터 기간 확인
- 결측치 및 중복 확인
- 기초 통계량 확인

```python
print("Interest Rate period:", interest["date"].min(), "~", interest["date"].max())
print("Exchange Rate period:", exchange["date"].min(), "~", exchange["date"].max())

print("\nInterest Rate describe:")
print(interest.describe())

print("\nExchange Rate describe:")
print(exchange.describe())
```

---

## 3. 시각화 (기초 확인)

```python
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.plot(pd.to_datetime(interest['date']), interest['interest_rate'], label="Interest Rate (%)", color="blue")
plt.plot(pd.to_datetime(exchange['date']), exchange['exchange_rate'], label="Exchange Rate (KRW/USD)", color="orange")
plt.legend()
plt.title("기준금리와 환율 추이 (Time Series)")
plt.show()
```

---

## 4. 데이터 품질 이슈 발견

- 환율 값이 약 **400 수준**으로 나와 실제 원/달러 환율(최소 900 이상)과 불일치
- 원인: `exchange_rate` 테이블에 **원/달러 지표가 누락**
- 확인: ECOS API의 환율 코드(`731Y001`) 내 여러 국가 통화가 포함되어 있었음
- 해결: **indicator를 "원/미국달러(매매기준율)"로 한정**하여 재수집

---

## 5. 상관 분석 (1차 확인)

```python
from scipy.stats import pearsonr

# 날짜 기준 병합
merged = pd.merge(interest, exchange, on="date")

# 피어슨 상관계수
r, p = pearsonr(merged["interest_rate"], merged["exchange_rate"])
print("Correlation:", r, "p-value:", p)
```

---

## 6. 단순 선형회귀

```python
from scipy.stats import linregress

slope, intercept, r_value, p_value, stderr = linregress(
    merged["interest_rate"], merged["exchange_rate"]
)

print("slope:", slope)
print("intercept:", intercept)
print("R^2:", r_value**2)
print("p-value:", p_value)
```

---

## 7. Lag 분석

```python
max_lag = 6
results = []
for lag in range(-max_lag, max_lag+1):
    shifted = merged["interest_rate"].shift(lag)
    corr = merged["exchange_rate"].corr(shifted)
    results.append({"lag": lag, "correlation": corr})

pd.DataFrame(results)
```

---

## 8. 간단한 결론

- 기준금리와 환율 간 **단순한 상관관계는 약함** (r ≈ -0.04, p > 0.05)
- 단순 선형회귀 역시 설명력(R²) 거의 없음
- 환율 변동은 금리 외에도 무역수지, 글로벌 경제 상황 등 다양한 요인에 의해 영향을 받음
- 그러나 데이터 품질 검증 과정에서 **잘못된 환율 데이터(400 수준)를 발견 → 원/달러 지표로 수정**한 경험은 데이터 엔지니어로서 중요한 문제 해결 과정임
